# 4. Challenge

![painting](https://th.bing.com/th/id/OIP.JEf5KjwvNHuo0-A_UnFBXAHaFj?pid=ImgDet&rs=1)

## Background information

This week's challenge will be a  bit different - instead of building a system for classification problem, we are going to develop simple GAN to generate art in Claude Monet's style!

Such system will consist of generator and discriminator models through which we will pass C. Monet's example artwork. After training such system, we will use trained generator to output new images that (in a perfect case) should be similar to the real artwork.

***As GAN models are quite complex, the training can take up to 30 minutes (depending on your device's specifications). Thus, you should use around 1000 images of the whole dataset***. This, of course, will have an impact on your output quality.

## Data

The training data containing Claude Monet's artwork can be accessed using the following [link](https://drive.google.com/file/d/1OCb3PjQTUvBtPMXpzS2Lsy_GT2i7D6HI/view?usp=sharing). It contains over 1100 images, but you should probably use only 1000 of them.

### Preprocessing

In [46]:
#Import libraries
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
import tensorflow as tf
import os
import PIL
import numpy as np
import cv2
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [223]:
#Define your image folder path
path = ___

After defining the image folder path, we need to import the image data. For such purpose, we are going to use PIL library (similar to opencv). Since some of you might not be familiar with such library, we have already written you a function that takes the folder path and the number of images you want to import and outputs images.

In [204]:
#Choose the dimensions of the image and leave function as it is
def import_images(path, number):
    image_file_list = os.listdir(path)
    
    pixels = []
    images = []
    
    for i in range(number):
        image = PIL.Image.open(path + '\\' + image_file_list[i], 'r')
        #Define your image dimensions
        width = ___
        
        image = image.resize((___, ___), PIL.Image.ANTIALIAS)
        pix = np.array(image.getdata())
        
        #Think about the number of channels
        pixels.append(pix.reshape(___, ___, ___))
        images.append(images)
    return np.array(pixels), images

In [224]:
#Import images and pixels and check the shape of your dataset
pixels,imgs = import_images(path, ___)
pixels.shape

(1000, 100, 100, 3)

### Building model

After a rather short preprocessing part, it's time to build our GAN model. As it has been mentioned in the theoretical part, the GAN model consists of generator and discriminator models. In the following code blocks, you will need to define these models and the GAN itself. To simplify the task, some part of the code is already written.

#### Discriminator
In the following function you will need to:
- Define the input shape (remember your image shape)
- After the first three layers, you will need to add the second convolutional block
- For the final two layers, flatten your inputs and pass through dense layer
- Define your optimizer and compile your model

In [206]:
def get_discriminator(in_shape = ___):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape),
        tf.keras.layers.LeakyReLU(alpha = 0.2),
        tf.keras.layers.Dropout(0.4),
        
        #Add the second conv block
        ___
        #Flatten your outputs
        ___
        #Pass through dense layer
        ___
    ])
    
    #Define optimizer
    optimizer = ___
    
    #Compile your model
    ___
    
    return model

#### Generator
The majority of code is already writte, so you will need to write two deconvolutional blocks containing LeakyReLU layer.

In [207]:
def get_generator(latent_dim):
    
    num_nodes = 128 * 25 * 25
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(num_nodes, input_dim = latent_dim),
        tf.keras.layers.LeakyReLU(alpha = 0.2),
        tf.keras.layers.Reshape((25, 25, 128)),
        
        #Add two deconvolutional layers containing LeakyReLU activation layer
        ___
        
        
        ___
 
        
        tf.keras.layers.Conv2D(3, (7,7) , padding='same')
    ])
    
    return model

#### GAN

Finally, combine these two models into GAN. In this code block, you will need to:
- Add generator and discriminator blocks to your model
- Define your optimizer
- Compile model

In [7]:
def get_gan(generator, discriminator):
    discriminator.trainable = False
    
    model = tf.keras.models.Sequential()
    
    #Add generator
    ___
    
    #Add discriminator
    ___
    
    #Define optimizer
    optimizer = ___
    
    #Compile your model
    ___
    
    return model

#### Additional functions

In addition to defining our model, we also need functions for formating the real image data, generating latent points that will be used as an input data to our generator ('clues' for our generator) and generating fake data itself. To simplify the problem, all these functions are already provided to you.

In [8]:
def real_data_gen(data, num_samples):
    
    idx = randint(0, data.shape[0], num_samples)
    X = data[idx]
    y = ones((num_samples, 1))
    
    return X, y

In [9]:
def latent_point_gen(latent_dim, num_samples):
    
    x = randn(latent_dim * num_samples)
    x = x.reshape(num_samples, latent_dim)
    
    return x

In [10]:
def fake_data_gen(generator, latent_dim, num_samples):
    
    x = latent_point_gen(latent_dim, num_samples)
    
    X = generator.predict(x)
    
    y = zeros((num_samples, 1))
    
    return X, y

### Model training

So far, we have defined our model and some additional functions that are going to be used for generating data. It's now time to actually train our system!

To make it easier for you, we also are providing all code for the train and performance summary (just to observe our training) functions. You will only need to define the number of epochs and batch.

As a whole, the train function operates in the following way:
- It takes input data and formates it
- Similarly, the fake data is generated
- After stacking fake and real data, we pass it through discriminator that outputs the performance

In [24]:
#In this block, only define the number of epochs and batch
def train(generator, discriminator, gan, data, latent_dim, num_epochs = ___, num_batch = ___):
    
    batch_per_epoch = int(data.shape[0] / num_batch)
    
    half_batch = int(num_batch / 2)
    
    for i in range(num_epochs):
        for j in range(batch_per_epoch):
            
            X_real, y_real = real_data_gen(data, half_batch)
            X_fake, y_fake = fake_data_gen(generator, latent_dim, half_batch)
            
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            
            discriminator_loss, _ = discriminator.train_on_batch(X, y)
            
            X_gan = latent_point_gen(latent_dim, num_batch)
            y_gan = ones((num_batch, 1))
            
            generator_loss = gan.train_on_batch(X_gan, y_gan)
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, batch_per_epoch, discriminator_loss, generator_loss))
        
        if (i+1) % 10 == 0:
            
            summarize_performance(i, generator, discriminator, data, latent_dim)
            clear_output()

In [11]:
def summarize_performance(epoch, generator, discriminator, data, latent_dim, num_samples=100):
    
    X_real, y_real = real_data_gen(data, num_samples)
    
    _, acc_real = discriminator.evaluate(X_real, y_real, verbose=0)
    
    x_fake, y_fake = fake_data_gen(generator, latent_dim, num_samples)
    
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    
    generator.save(filename)

Lastly, let's train apply all functions and train the system (it might take 15-30 minutes to train your model).

In [225]:
latent_dim = 100

#Import discriminator
___

#Import generator
___

#Import gan
___

train(generator, discriminator, gan, np.array(pixels), latent_dim)

### Generating art

Finally, we can use this model to generate example images.

In [ ]:
#model = generator

#model = tf.keras.models.load_model(r"C:/Users/marty/Desktop/ML learning/AI_SOC_TUTORIALS/WEEK_8/Challenge/generator_model_100.h5")

latent_points = latent_point_gen(100, 1)

X = model.predict(latent_points)

array = np.array(X.reshape(100,100,3), dtype=np.uint8)

new_image = PIL.Image.fromarray(array)

plt.imshow(new_image)